# Práctica Métodos de Simulación - Parte 3

## Elena Rivas, Teresa Grau, Ignacio Casso

## Enunciado

**El problema de las n reinas.**

El problema de las n-reinas consiste en colocar n reinas en un tablero de ajedrez de
n x n de tal manera que no sea posible que dos reinas se capturen entre si, es decir,
que no estén en la misma fila, ni en la misma columna ni en la misma diagonal. Se
dice que hay una colisión si hay dos reinas que se pueden capturar entre si.

Se trata pues de encontrar una configuración (elegir las n celdas donde colocar a las
reinas) que minimice el número total de colisiones. Obtener la solución óptima para
n=9.

Mostrar gráficamente la evolución del valor de la función fitness y del parámetro
temperatura considerados a lo largo de las iteraciones de la metaheurı́stica.

**Enfriamiento simulado**

El Enfriamiento simulado es una técnica de optimización combinatorial que se usa para afrontar problemas de gran complejidad matemática, de modo que se obtengan soluciones cercanas a la óptima. 
Se fundamenta en el proceso físico de calentamiento de un sólido, seguido por un enfriamiento hasta lograr un estado cristalino con una estructura perfecta. Durante este proceso, la energía libre del sólido es minimizada. 

"A una temperatura T, en el estado con nivel de energía Eì, se genera el estado siguiente j a través de de una pequeña perturbación. Si la diferencia de energía Ej – Ei es menor o igual a cero, el estado j es aceptado. Si la diferencia de energía es mayor que cero, el estado j es aceptado con una probabilidad P, donde KB es la constante de Boltzmann. Si la disminución de la temperatura es hecha de manera paulatina, el sólido puede alcanzar el estado de equilibrio en cada nivel de temperatura." 

## Planteamiento

#### El problema de las n reinas se puede plantear de dos maneras: 

- A) ubicando n reinas de manera que se minimicen las colisiones 
- B) maximizar el número de reinas en el tablero sujeto a la restricción de que no existan colisiones

## Soluciones

En esta práctica nos vamos a centrar en el planteamiento decrito anteriormente como A), para el que queremos minimizar el número de colisiones entre las reinas. Para éste proponemos dos implementaciones distintas para el algoritmo de enfriamiento simulado, que se diferencian en el espacio de búsqueda y en la codificación de las soluciones:
- A1)  En esta solución consideramos todo el espacio de búsqueda, es decir, todas las posibles configuraciones del tablero con $n$ reinas en él. 
- A2) En este caso tenemos en cuenta que por necesidad debe haber una reina en cada fila y otra en cada columna, y codificamos las soluciones como permutaciones de $n$ elementos, reduciendo el espacio de soluciones de un cardinal de $(n^2)^n$ a uno de $n!$. 

Para cada una de ellas se exploran distintas elecciones de parámetros y los resultados que se obtienen.

## Caso A1) Consideramos todas las posibles configuraciones del tablero con n reinas.

### Codificación de las soluciones

Codificaremos las soluciones como listas de 9 posiciones, siendo cada posición una tupla de 2 elementos. Para evitar redundancia en la codificación, se impone que esa lista esté ordenada (lexicográficamente), y para evitar soluciones no factibles, se prohiben los elementos repetidos en una lista.

In [13]:
from random import random, randint

### Función objetivo

La función objetivo a minimizar será el número de colisiones entre reinas.

In [14]:
def num_collisions(positions):
    
    if positions==[]:
        return 0
    else:
        queen, positions2 = positions[0], positions[1:]
        num_colls = 0
        for queen2 in positions2:
            if collision(queen,queen2):
                num_colls += 1
        return num_colls + num_collisions(positions2)
    
def collision(queen1,queen2):
    i1,j1 = queen1
    i2,j2 = queen2
    
    if i1==i2 or j1==j2 or abs(i1-i2)==abs(j1-j2):
        return True
    else:
        return False

In [15]:
def numCollisionsFitness(sol,n):
    return num_collisions(sol)

### Entorno de un punto en el espacio de búsqueda

Proponemos los siguientes entornos para una solución:

La primera, las soluciones obtenidas al mover una reina una posición en cualquier dirección.

In [16]:
moves = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]

def oneMoveRandomNeighbor(sol,n):
    queens = sol[:] # copy
    i = randint(0,n-1)
    j = randint(0,7)
    qx,qy = queens[i]
    mx,my = moves[j]
    nqx,nqy = qx+mx, qy+my
    if 0<nqx and 0 < nqy and n >= nqx and n>=nqy:
        new_queen = (nqx,nqy)
    else:
        new_queen = qx,qy
    queens[i] = new_queen
    return queens

En esta solución no se restringe el problema de que un determinado movimiento nos saque del tablero o que de que haya 2 reinas en la misma posición.

### Valor inicial de la temperatura

La temperatura debe ser tal en el proceso que permita una adecuada exploración en las diferentes etapas del enfriamiento simulado. Existen diferentes formas para calcular la temperatura inicial, siendo una de ellas simular el proceso para la primera cadena de Markov.  

Para este caso vamos a proponer los siguientes valores iniciales de temperatura:

In [17]:
def standardInitialTemp():
    return 0.9

### Variación de la temperatura

Proponemos las siguientes actualizaciones de temperatura:

En primer lugar, la propuesta en las transparencias, con $L=10,~ \alpha = 0.95$

In [18]:
def updateTempStandard(T,i):
    if i%10 == 0:
        newT = 0.95*T
    else:
        newT = T
    return newT

### Probabilidad de aceptación de nuevas soluciones

En este caso proponemos la estándar:

**Duda:** No debería estar la probabilidad entre 0 y 1?

In [19]:
from math import exp

In [20]:
def prob_aceptacion_standard(f1,f,T):
    x = -(f1-f)/T
    return exp(x)

### Criterio de parada

Se pueden usar varios criterios de parada:
    
- Parar si se ha encontrado la solución óptima     
- Fijar un número determinado de niveles de temperatura, usualmente entre 6 y 50. 
- Terminar el procesos si la función objetivo no mejora para varios niveles consecutivos. 
- Parar si no se cumple con un número mínimo de aceptaciones en el nivel de temperatura.

El caso que vamos a utilizar es en el que la parada se produce cuando se encuentra la solución óptima:

In [21]:
def solutionFound(f,_):
    return f==0

### Configuración inicial

De la configuración inicial depende la rapidez para encontrar una solución. 
Por ejemplo para una inicialización hecha de forma aleatoria, se tiene que alrededor de la mitad de las reinas están en colisión. Existen estrategias para inicializar la configuración de tal manera que se reduzca el número de colisiones al comienzo del proceso. 

La solución que planteamos es una de las más simples a través de la que se colocan todas las reinas en la primera fila.

In [22]:
def initialSolSameRow(n):
    return [(1,i+1) for i in range(n)]

In [23]:
initialSols=[initialSolSameRow]

### Algoritmo

In [24]:
from random import random

In [27]:
def queens(n):
    sol = initialSol(n)
    f = fitness(sol,n)
    best_sol = sol
    best_f = f
    T = initialTemp()
    fitness_values=[f]
    temp=[T]
    i=1    
    while not criterio_parada(best_f,i):
        new_sol = random_neighbor(sol,n)
        new_f = fitness(new_sol,n)
        if new_f < f:
            sol = new_sol
            f = new_f
            if f < best_f:
                best_sol=sol
                best_f = f
        else:
            u = random()
            p = prob_aceptacion(new_f,f,T)
            if p > u:
                sol = new_sol
                f = new_f
        update_temp(T,i)
        i += 1
    return sol

In [26]:
def queens(n):
    sol = initialSol(n)
    f = fitness(sol,n)
    best_sol = sol
    best_f = f
    T = initialTemp()
    fitness_values=[f]
    temp=[T]    
    i=1    
    while not criterio_parada(best_f,i):
        new_sol = random_neighbor(sol,n)
        new_f = fitness(new_sol,n)
        if new_f < f:
            sol = new_sol
            f = new_f
            if f < best_f:
                best_sol=sol
                best_f = f
        else:
            u = random()
            p = prob_aceptacion(new_f,f,T)
            if p > u:
                sol = new_sol
                f = new_f         
        T = update_temp(T,i)  
        print i,T,f
        i += 1        
    return sol
  

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i,T,f)? (<ipython-input-26-c32c7f6bba3e>, line 26)

### Resultados

Para probar la implementación del algoritmo se ha resuelto el problema de las N reinas para distintos valores de N. Para cada tablero se ha lanzado 5 veces el programa. 

In [33]:
initialSol=initialSolSameRow
fitness = numCollisionsFitness
initialTemp = standardInitialTemp
criterio_parada = solutionFound
random_neighbor = oneMoveRandomNeighbor
update_temp = updateTempStandard
prob_aceptacion = prob_aceptacion_standard

print(initialSol[0])

TypeError: 'function' object is not subscriptable

In [29]:
queens(9)

[(8, 3), (1, 8), (7, 6), (4, 1), (5, 7), (9, 5), (2, 2), (6, 9), (3, 4)]

In [30]:
import numpy as np
import matplotlib.pyplot as plt

Se muestra en el gráfico siguiente la evolución del valor de la función fitness a lo largo de las iteraciones.

In [31]:
plt.plot(i, f)
plt.show()

NameError: name 'i' is not defined

En el siguiente gráfico se muestra la evolución del parámetro temperatura en las diferentes iteraciones.

In [ ]:
plt.plot(i, T)
plt.show()

Se muestra en la tabla siguiente el promedio del número de iteraciones en las que se alcanzó la solución del problema.
 

En la siguiente tabla aparecen los datos para las 10 ejecuciones con diferentes valores de N.
Se observa que el número de iteraciones crece de forma lineal con el tamaño del tablero. 
El tiempo de cómputo a su vez es proporcional al número de iteraciones.
El tiempo para iteración se mantiene constante al aumentar la dimensión del tablero, debido a que se analizan 8 diagonales en cada iteración sin importar el tamaño del problema. 

## Caso A2) Tenemos en cuenta que debe haber una reina en cada fila y otra en cada columna, y reducimos el problema a una permutación.

Puesto que dos reinas colisionan cuando están en la misma columna, fila o diagonal esto significa que el problema a solucionar pasar por evitar que las reinas vayan en una misma fila, es decir, que a cada reina le corresponda una fila de manera que la reina k se sitúa en la fila k. Y que del mismo modo la columna en la que se coloca tendrá que ser también diferente para cada reina. De este modo el problema se convierte en un problema de permutación. 
 
Si R(i) es el índice de la columna donde se localiza la reina i, entonces la configuración del tablero se describe con el vector R. Para el caso del tablero 9x9 del vector de configuración R es [1, 3, 5, 7, 9, 2, 4, 6, 8]. 

Por tanto si a cada reina se le asigna una fila y una columna que no comparte con otra, no existen colisiones entre reinas por filas o columnas, entonces el problema que debe analizarse es principalmente la configuración para las diagonales. Así si dos o más reinas comparten el mismo índice para una diagonal, entonces existe una colisión.

La configuración inicial es creada según ... y la temperatura inicial se calcula ... . 
Se genera una nueva configuración a través del intercambio de columnas entre dos reinas escogidas de forma aleatoria, pero dándole mayor probabilidad a las reinas que están en colisión de acuerdo con una lista que se va actualizando. 